In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"

import cudf

In [2]:
df = cudf.read_parquet(
    './data/xgb_train_x.parquet'
)
#df = cudf.read_parquet('../../data/test.parquet')

In [3]:
df1 = df.groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid'})

In [4]:
df2 = df[['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid'})

In [5]:
df3 = df[['session', 'ts']].groupby(['session']).agg(['min', 'max']).reset_index()
df3.columns = ['session', 'min', 'max']
df3['diff'] = df3['max']-df3['min']
df3.drop(['min', 'max'], axis=1, inplace=True)

In [6]:
df4 = df.sort_values(['session', 'ts'], ascending=[True, False])
df4['ts_2'] = df4.groupby(['session']).ts.shift(1)
df4['diff_'] = df4['ts_2']-df4['ts']
df4['diff_'] = ((df4['diff_']/1000)>(4*60*60)).fillna(0).astype('int8')
df4['new_session'] = df4.groupby(['session']).diff_.cumsum()
df4.drop(['ts_2', 'diff_'], axis=1, inplace=True)
df5 = df4[['session', 'new_session']].drop_duplicates().groupby(['session']).agg(['max']).reset_index()
df5.columns  = ['session', 'num_sessions']

df6 = df4[df4['new_session']==0].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_fs'})
df7 = df4[df4['new_session']==0][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_fs'})

In [7]:
df8 = df[df['type']=='clicks'].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_cl'})
df9 = df[df['type']=='carts'].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_ca'})
df10 = df[df['type']=='orders'].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_co'})

In [8]:
df11 = df[df['type']=='clicks'][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_cl'})
df12 = df[df['type']=='carts'][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_ca'})
df13 = df[df['type']=='orders'][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_co'})

In [9]:
df14 = df4[df4['type']=='clicks'].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_cl_fs'})
df15 = df4[df4['type']=='carts'].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_ca_fs'})
df16 = df4[df4['type']=='orders'].groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_aid_co_fs'})

In [10]:
df17 = df4[df4['type']=='clicks'][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_cl_fs'})
df18 = df4[df4['type']=='carts'][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_ca_fs'})
df19 = df4[df4['type']=='orders'][['session', 'aid']].drop_duplicates().groupby(['session']).aid.count().reset_index().rename(columns={'aid': 'num_u_aid_co_fs'})

In [11]:
dfs = [
    df2,
    df3,
    df5,
    df6,
    df7,
    df8,
    df9,
    df10,
    df11,
    df12,
    df13,
    df14,
    df15,
    df16,
    df17,
    df18,
    df19
]

In [12]:
for df_tmp in dfs:
    df1 = df1.merge(
        df_tmp,
        how='left',
        on=['session']
    ).fillna(0)

In [13]:
df1.to_parquet('./data/train_sessions.parquet')
#df1.to_parquet('./data/sub_sessions.parquet')